In [ ]:
# import libraries
# read files
# check datatypes
# check for categorical value misspell
# Standardize categorical values (spelling, capitalization etc)
# Check for missing values
# Crosscheck all numerical values
# Check and modify column data types
# check for duplicates if necessary
# Use visualization for outliers
# Remove irrelevant columns
# Convert units to a consistent format